# Análise do 7GHz

- Gera o gráfico do evento em 2.6, 4.9, 7, 8.8, 15.4GHz
- Gera o espectro rádio
- Gera a página para o dashboard

In [ ]:
%matplotlib ipympl

import datetime as dt
import os
import platform

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.dates import num2date, DateFormatter
from scipy.io import readsav
from sunpy.time import TimeRange
from sunpy.timeseries import TimeSeries
from sunpy.net import Fido, attrs

from rstnpy import RSTN
from noaareport import NoaaReport

from utils import (
    onclick, posicao, calculo_de_indice, load_dados,
    remove_background, CAMINHO_ABSOLUTO, get_correct_goes_index
)

In [ ]:
dia = "09"
mes = "04"
ano = "2002"
df, time, diretorio = load_dados(dia, mes, ano)

## Configure o tamanho do gráfico

In [ ]:
plt.rcParams['figure.figsize'] = 7,5

## Ordem de seleção dos pontos para remover o background:

- Os dois primeiros pontos são usados para selecionar o evento
- Os dois últimos pontos são usados para remover o background

In [ ]:
fig_dia_7ghz, axis = plt.subplots()

axis.plot(df)
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

fig_dia_7ghz.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
dados_da_media = remove_background(df)

indice_ig1 = dados_da_media["indice_inicio_evento"]
indice_ig2 = dados_da_media["indice_fim_evento"]
media_final_r = dados_da_media["medias"]["R"]
media_final_l = dados_da_media["medias"]["L"]
inicio_flare = dados_da_media["inicio_flare"]
fim_flare = dados_da_media["fim_flare"]

In [ ]:
fig_R, axis = plt.subplots()
# Usa o mesmo index do dataframe para o eixo x.
axis.plot(df["R"][inicio_flare:fim_flare])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

fig_L, axis = plt.subplots()
# Usa o mesmo index do dataframe para o eixo x.
axis.plot(df["L"][inicio_flare:fim_flare])
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

## Gráfico do Evento, I e V

In [ ]:
LM_normalizado = df["L_normalizado"][inicio_flare:fim_flare]
RM_normalizado = df["R_normalizado"][inicio_flare:fim_flare]

RL_V = df["R_normalizado"][inicio_flare:fim_flare] - df["L_normalizado"][inicio_flare:fim_flare]
RL_I = df["L_normalizado"][inicio_flare:fim_flare] + df["R_normalizado"][inicio_flare:fim_flare]

fig_IV, (axis_top, axis_mid, axis_bottom) = plt.subplots(3)

# Usa o mesmo index do dataframe para o eixo x.
axis_top.plot(LM_normalizado, label="L")
axis_top.plot(RM_normalizado, label="R")
axis_top.set_title("Analise 7 GHz")
axis_top.legend()
axis_top.set_ylabel("Flux (SFU)")
axis_top.yaxis.grid(color='gray')

axis_mid.plot(RL_I)
axis_mid.set_ylabel("I (SFU)")
axis_mid.yaxis.grid(color='gray')

axis_bottom.plot(RL_V)
axis_bottom.set_ylabel("V (SFU)")
axis_bottom.yaxis.grid(color='gray')

axis_top.xaxis.set_major_formatter(DateFormatter('%H:%M'))
axis_mid.xaxis.set_major_formatter(DateFormatter('%H:%M'))

axis_bottom.xaxis.set_major_formatter(DateFormatter('%H:%M'))
pico_7ghz = max(RL_I)

## Salva o gráfico

In [ ]:
nome_do_arquivo = "I-V-Flux(SFU)"
fig_IV.savefig(CAMINHO_ABSOLUTO.joinpath(diretorio).joinpath(nome_do_arquivo))

## Pico 7GHz

In [ ]:
fig_pico_7ghz, axis = plt.subplots()

# Usa o index do dataframe para o eixo x.
axis.plot(RM_normalizado, label="R")
axis.plot(LM_normalizado, label="L")

axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

fig_pico_7ghz.canvas.mpl_connect('button_press_event', onclick)
plt.legend()

In [ ]:
print(posicao[-1])
posicao_pico_7ghz = posicao[-1]
horario_pico_7ghz = num2date(posicao_pico_7ghz[0])
print("Pico:", horario_pico_7ghz)

descarte, tempo_inicial = calculo_de_indice(RL_I, horario_pico_7ghz)
pico_7ghz = RL_I[tempo_inicial]

descarte, tempo_inicial = calculo_de_indice(RM_normalizado, horario_pico_7ghz)
pico_r_7ghz = RM_normalizado[tempo_inicial]

descarte, tempo_inicial = calculo_de_indice(LM_normalizado, horario_pico_7ghz)
pico_l_7ghz = LM_normalizado[tempo_inicial]

## RSTN

### Pega os dados do RSTN do dia

In [ ]:
rstn_station = "Sagamore Hill"
rstn = RSTN(ano, mes, dia, "dados_rstn", rstn_station)
filename = rstn.downloader.download_file()
rstn.decompress_file(filename)
rstn_data = rstn.create_dataframe()

### Descarta as frequências que não serão usadas

In [ ]:
del rstn_data["245"]
del rstn_data["410"]
del rstn_data["610"]
del rstn_data["1415"]

## Ordem de seleção dos pontos para remover o background:

- Os dois primeiros pontos são usados para selecionar o evento
- Os dois últimos pontos são usados para remover o background

In [ ]:
ax = rstn_data.plot()
fig_rstn = ax.get_figure()
fig_rstn.canvas.mpl_connect('button_press_event', onclick)

### Remove background

In [ ]:
posicao_inicial_rstn = posicao[-2]
posicao_final_rstn = posicao[-1]

p1 = num2date(posicao_inicial_rstn[0])
p2 = num2date(posicao_final_rstn[0])

descarte, tempo_inicial = calculo_de_indice(rstn_data, p1)
descarte, tempo_final = calculo_de_indice(rstn_data, p2)

medias = remove_background(rstn_data, True)

fig_rstn, axis = plt.subplots()

plt.title("RSTN Normalizado")
for column in rstn_data.columns:
    if len(column) > 11:
        axis.plot(rstn_data.index, rstn_data[column], label=column)

axis.xaxis.set_major_formatter(DateFormatter("%H:%M"))
plt.legend()

### Corta o gráfico somente na parte do evento

In [ ]:
posicao_inicial_rstn = posicao[-4]
posicao_final_rstn = posicao[-3]

posicao_inicial = num2date(posicao_inicial_rstn[0])
posicao_final = num2date(posicao_final_rstn[0])

descarte, tempo_inicial = calculo_de_indice(rstn_data, posicao_inicial)
descarte, tempo_final = calculo_de_indice(rstn_data, posicao_final)

### Pega os valores do pico nas frequências do RSTN

É importante pegar somente a parte do evento na célula anterior pra que o pico possa ser pego da forma correta.

In [ ]:
picos = {
    "2.695": 0,
    "4.995": 0,
    "7.0": pico_7ghz,
    "8.8": 0,
    "15.4": 0
}

fig_rstn_normalizado, axis = plt.subplots()

axis.set_title("RSTN Normalizado")
for column in rstn_data.columns:
    if len(column) < 11:
        continue
        
    if "2695" in column:
        picos["2.695"] = max(rstn_data[column][tempo_inicial:tempo_final])
    elif "4995" in column:
        picos["4.995"] = max(rstn_data[column][tempo_inicial:tempo_final])
    elif "8800" in column:
        picos["8.8"] = max(rstn_data[column][tempo_inicial:tempo_final])
    elif "15400" in column:
        picos["15.4"] = max(rstn_data[column][tempo_inicial:tempo_final])
        
    axis.plot(rstn_data.index[tempo_inicial:tempo_final],
              rstn_data[column][tempo_inicial:tempo_final],
              label=column)

print(picos)
axis.xaxis.set_major_formatter(DateFormatter("%H:%M"))
plt.legend()

In [ ]:
horario_do_pico = str(horario_pico_7ghz)[10:16]

## Espectro rádio

In [ ]:
# Colocar as frequencias em ordem.
picos_ordenados = [float(x) for x in picos.keys()]

fig_espectro, axis = plt.subplots()

for column in picos_ordenados:
    axis.plot(column, picos[str(column)], '-o', label=column)

axis.set_xlabel("Frequência")
axis.set_ylim([0, 500])
axis.set_xlim([0.2, 20])
axis.yaxis.grid(color='gray')
axis.set_title("Espectro Rádio" + horario_do_pico)
plt.legend()

## Salva o gráfico

In [ ]:
nome_do_arquivo = "espectro"
fig_espectro.savefig(CAMINHO_ABSOLUTO.joinpath(diretorio).joinpath(nome_do_arquivo))

## GOES

In [ ]:
comeco_flare = str(df.index[inicio_flare])
fim_flare = str(df.index[fim_flare])

comeco_flare = comeco_flare[0:19].replace('-', '/')
fim_flare = fim_flare[0:19].replace('-', '/')

tr = TimeRange(comeco_flare, fim_flare)

results = Fido.search(attrs.Time(tr), attrs.Instrument("XRS"))
files = Fido.fetch(results)
goes = TimeSeries(files)
goes = goes.to_dataframe()
comeco, fim = get_correct_goes_index(goes.index, comeco_flare, fim_flare)
goes = goes[comeco:fim]

In [ ]:
fig, axis = plt.subplots()

# Usa o mesmo index do dataframe para o eixo x.
axis.semilogy(goes.index, goes["xrsa"], label=r"0.5--4.0 $\AA$", color="b", linewidth=2)
axis.semilogy(goes.index, goes["xrsb"], label=r"1.0--8.0 $\AA$", color="r", linewidth=2)
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))
axis.set_title("GOES Xray Flux")
axis.set_axisbelow(True)
axis.yaxis.grid(color='gray')
axis.set_ylabel("Watts $m^{-2}$")
axis.set_xlabel(ano + "-" + mes + "-" + dia)
axis.set_ylim([1e-9, 1e-2])

plt.legend()

ax2 = axis.twinx()
ax2.set_yscale("log")
ax2.set_ylim(1e-9, 1e-2)
ax2.set_yticks((1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2))
ax2.set_yticklabels((' ', 'A', 'B', 'C', 'M', 'X', ' '))

fig_goes = ax2.get_figure()

## Salva o gráfico

In [ ]:
nome_do_arquivo = "GOES"
fig_goes.savefig(CAMINHO_ABSOLUTO.joinpath(diretorio).joinpath(nome_do_arquivo))

## Região Ativa

In [ ]:
comeco_flare = comeco_flare.replace('/', '-')
fim_flare = fim_flare.replace('/', '-')

caminho_eventos = "Eventos/" + ano + "_events/"
report = NoaaReport(ano, mes, dia, caminho_eventos)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(report.get_dataframe())

In [ ]:
ar = "None"

## Grau de polarização

In [ ]:
fig_7ghz, axis = plt.subplots()

# Usa o mesmo index do dataframe para o eixo x.
axis.plot(LM_normalizado, label="L")
axis.plot(RM_normalizado, label="R")
axis.xaxis.set_major_formatter(DateFormatter('%H:%M'))

axis.legend()
axis.set_ylabel("Flux (SFU)")
axis.set_xlabel("Horário")
plt.title("Evento 7GHz")

fig_7ghz.canvas.mpl_connect('button_press_event', onclick)

## Salva o gráfico

In [ ]:
nome_do_arquivo = "grafico_normalizado"
fig_7ghz.savefig(CAMINHO_ABSOLUTO.joinpath(diretorio).joinpath(nome_do_arquivo))

In [ ]:
posicao_inicial= num2date(posicao[-1][0])
indice_de_y1 = calculo_de_indice(df, posicao_inicial)

pico_R = pico_r_7ghz
pico_L = pico_l_7ghz

ponto_do_r_inicial = indice_de_y1[0][2]
ponto_do_r_inicial = float("{:.3f}".format(ponto_do_r_inicial))
print('Ponto do R inicial:', ponto_do_r_inicial)

ponto_do_r_no_pico = float(str(pico_R)[0:7])
ponto_do_r_no_pico = float("{:.3f}".format(ponto_do_r_no_pico))
print('Ponto do R no pico:', ponto_do_r_no_pico)

ponto_do_l_inicial = indice_de_y1[0][3]
ponto_do_l_inicial = float("{:.3f}".format(ponto_do_l_inicial))
print('Ponto do L incial:', ponto_do_l_inicial)

ponto_do_l_no_pico = float(str(pico_L)[0:7])
ponto_do_l_no_pico = float("{:.3f}".format(ponto_do_l_no_pico))
print('Ponto do L no pico:', ponto_do_l_no_pico)

horario_do_pico = num2date(posicao_pico_7ghz[0])
tempo = str(horario_do_pico)[0:26]
print('Tempo:', tempo)

### Cálculo do I e V

In [ ]:
I = ((ponto_do_r_no_pico - ponto_do_r_inicial) + (ponto_do_l_no_pico - ponto_do_l_inicial))

I = float("{:.3f}".format(I))
print("Cálculo do I:", I)

V = ((ponto_do_r_no_pico - ponto_do_r_inicial) - (ponto_do_l_no_pico - ponto_do_l_inicial))
V = float("{:.3f}".format(V))
print("Cálculo do V:", V)

### Grau de polarização.

In [ ]:
GP = (V / I) * 100
GP = float("{:.3f}".format(GP))
print('Grau de Polarização:', GP)

## Geração de página

In [ ]:
import matplotlib
from matplotlib import gridspec

font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 8}

matplotlib.rc('font', **font)

ut = "Universal Time"

plt.rcParams['figure.figsize'] = 10,7

fig_pagina, axes = plt.subplots(3, 3)

if len(dia) == 1:
    dia = "0" + dia
if len(mes) == 1:
    mes = "0" + mes

gs = gridspec.GridSpec(6, 6, figure=fig_final)
gs.update(wspace=1.4, hspace=2)

fig_pagina.suptitle("Event {:%B %d, %Y}".format(horario_do_pico), fontsize=14)

ax1 = plt.subplot(gs[0:2, :3])
ax2 = plt.subplot(gs[2:4, :3])
ax3 = plt.subplot(gs[4:, :3])
ax4 = plt.subplot(gs[0:2, 3:])
ax5 = plt.subplot(gs[2:4, 3:])
ax6 = plt.subplot(gs[4:, 3:])

## Evento 7GHz

In [ ]:
ax1.plot(LM_normalizado, label="L")
ax1.plot(RM_normalizado, label="R")
ax1.set_title("Event - 7 GHz - {}/{}/{}".format(mes, dia, ano[2:]))
ax1.set_ylabel("Flux (SFU)")
ax1.set_xlabel(ut)
ax1.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax1.yaxis.grid(color='gray')
ax1.legend()

## R + L (I)

In [ ]:
ax2.plot(RL_I)
ax2.set_ylabel("I [R+L] (SFU)")
ax2.set_xlabel(ut)
ax2.yaxis.grid(color='gray')
ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))

## R - L (V)

In [ ]:
ax3.plot(RL_V)
ax3.set_ylabel("V [R-L] (SFU)")
ax3.set_xlabel(ut)
ax3.yaxis.grid(color='gray')
ax3.xaxis.set_major_formatter(DateFormatter('%H:%M'))

## Evento em Raios x

In [ ]:
ax4.semilogy(goes.index, goes["xrsa"], label=r"0.5--4.0 $\AA$", color="b", linewidth=2)
ax4.semilogy(goes.index, goes["xrsb"], label=r"1.0--8.0 $\AA$", color="r", linewidth=2)
ax4.xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax4.set_title("GOES Xray Flux")
ax4.set_axisbelow(True)
ax4.yaxis.grid(color='gray')
ax4.set_ylabel("Watts $m^{-2}$")
ax4.set_xlabel(ut)
ax4.set_ylim([10**-9, 10**-2])
ax4.legend()

ax4_2 = ax4.twinx()
ax4_2.set_yscale("log")
ax4_2.set_ylim(1e-9, 1e-2)
ax4_2.set_yticks((1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2))
ax4_2.set_yticklabels((' ', 'A', 'B', 'C', 'M', 'X', ' '))

## Espectro rádio

In [ ]:
for column in picos_ordenados:
    ax5.plot(column, picos[str(column)], '-o', label=column)

ax5.set_ylabel("SFU")
ax5.set_xlabel("Frequency (GHz)")
ax5.set_ylim([0, 500])
ax5.set_xlim([0.2, 20])
ax5.yaxis.grid(color='gray')
ax5.set_title("Espectro Rádio - {:%H:%M}".format(horario_do_pico))
ax5.legend()

## Evento RSTN

In [ ]:
ax6.set_title("RSTN - " + rstn_station)
for column in rstn_data.columns:
    if len(column) < 11:
        continue
        
    if "2695" in column:
        ax6.plot(rstn_data.index[tempo_inicial:tempo_final],
                rstn_data[column][tempo_inicial:tempo_final],
                label="2.695")
    elif "4995" in column:
        ax6.plot(rstn_data.index[tempo_inicial:tempo_final],
                 rstn_data[column][tempo_inicial:tempo_final],
                 label="4.995")
    elif "8800" in column:
        ax6.plot(rstn_data.index[tempo_inicial:tempo_final],
                 rstn_data[column][tempo_inicial:tempo_final],
                 label="8.8")
    elif "15400" in column:
        ax6.plot(rstn_data.index[tempo_inicial:tempo_final],
                 rstn_data[column][tempo_inicial:tempo_final],
                 label="15.4")

ax6.xaxis.set_major_formatter(DateFormatter("%H:%M"))
ax6.legend()

## Salva o gráfico

In [ ]:
nome_do_arquivo = "pagina"
fig_pagina.savefig(CAMINHO_ABSOLUTO.joinpath(diretorio).joinpath(nome_do_arquivo))

In [ ]:
resultados_finais = np.transpose([[tempo], [ponto_do_r_inicial], [ponto_do_l_inicial],
                                  [ponto_do_r_no_pico], [ponto_do_l_no_pico], [GP],
                                  [I], [V], [str(ar)]])

dados_finais = pd.DataFrame(resultados_finais, columns = ['Time','PR','PL','PRP','PLP','GP','I','V', 'AR'])

nome_do_arquivo = CAMINHO_ABSOLUTO.joinpath("dados_finais/dados.csv")

dados_finais.to_csv(nome_do_arquivo, header=None, mode='a', index=False)